# 4.0 Exploring the Breakfast Dataset

Going through the breakfast dataset to see what the data look like.

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [40]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,matplotlib,seaborn,jupyterlab,lab

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Sat Feb 15 2020 17:45:53 

CPython 3.6.10
IPython 7.12.0

torch 1.3.1
torchvision 0.1.8
cv2 3.4.2
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.12.g117bd38.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.4.0-173-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : 117bd3817284447482a5ed6253ee55e5dd76cece
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [6]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [7]:
# Set the matplotlib mode
%matplotlib inline

## Set the GPU

Make sure we aren't greedy.

In [8]:
!nvidia-smi

Sat Feb 15 16:57:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   21C    P8     8W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   23C    P8     8W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [9]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## Imports

In [36]:
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [11]:
# Constants to be used throughout the package
%aimport lab.index
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import DIR_BREAKFAST_VIDEOS, DIR_COARSE_SEG, DIR_FINE_SEG

## Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [12]:
sns.set()

## Directory Contents

In [18]:
contents = list(DIR_BREAKFAST_VIDEOS.iterdir())

In [20]:
print(len(contents))

52


In [21]:
contents[0]

PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39')

In [23]:
contents[0].is_dir()

True

In [24]:
video_contents_0 = list(contents[0].iterdir())

In [25]:
len(video_contents_0)

5

In [26]:
video_contents_0

[PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/webcam01'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/webcam02'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/stereo'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam02')]

In [27]:
for file in video_contents_0:
    print(file.stem, file.is_dir())

cam01 True
webcam01 True
webcam02 True
stereo True
cam02 True


In [32]:
cam01 = video_contents_0[0]
list(cam01.iterdir())

[PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_salat.avi'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_pancake.avi.labels'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_tea.avi'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_milk.avi.labels'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_sandwich.avi.labels'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_scrambledegg.avi.labels'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_tea.avi.labels'),
 PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_pancake.avi'),
 Pos

In [34]:
cam01_example_video = list(cam01.glob('*.avi'))[0]
cam01_example_video

PosixPath('/home/abdullah/work/labwork/data/external/breakfast/BreakfastII_15fps_qvga_sync/P39/cam01/P39_salat.avi')